In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, f1_score, roc_auc_score, roc_curve, ConfusionMatrixDisplay
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import os
import sys
import shap
os.getcwd()

# Read in data

In [ ]:
df_raw_large = pd.read_parquet('../../../../dataset/sampled-dataset/raw/sample-large.parquet')
df_pro_large=o_small = pd.read_parquet('../../../../dataset/sampled-dataset/processed/sample-large.parquet')
df_large = df_raw_large.join(df_pro_large)
df_large

# EDA

In [ ]:

numerical_features = [
    'routing_length', 'word_count', 'readable_proportion',
    'whitespace_ratio', 'alphabet_proportion', 'grammar_error_rate',
    'english_french_proportion', 'url_count'
]
categorical_features = [
    'dkim_result', 'spf_result', 'dmarc_result', 'html_parsing_error'
]
binary_features = [
    'is_multipart', 'attachments_present',
    'dmarc_authentication_present', 'dkim_sender_domains_match',
    'to_from_addresses_match', 'sender_email_spf_match',
    'non_ascii_present', 'hidden_text_present',
    'ip_addr_urls', 'http_urls_present', 'url_at_symbol',
    'url_port_number', 'any_long_urls', 'url_multiple_subdomains'
]

target_col = 'target_1'

model_features = numerical_features + categorical_features + binary_features
X = df_large[model_features].copy()
y = df_large[target_col].map({'benign': 0, 'malicious': 1}) 

# Numerical features distribution

In [ ]:
num_feats = len(numerical_features)
cols = 4
rows = (num_feats + cols - 1) // cols

fig, axes = plt.subplots(rows, cols, figsize=(cols*4, rows*3))
axes = axes.flatten()
for i, col in enumerate(numerical_features):
    temp = X.copy()
    temp['target'] = y
    sns.histplot(
        data=temp, x=col, hue='target', kde=True, bins=30, 
        stat='count', palette='Set1', alpha=0.6, ax=axes[i]
    )
    axes[i].set_title(f"{col} by target")
for i in range(len(numerical_features), len(axes)):
    fig.delaxes(axes[i])
plt.tight_layout()
plt.show()

# categorical features distribution

In [ ]:
cat_feats = len(categorical_features)
cols = 4
rows = (cat_feats + cols - 1) // cols

fig, axes = plt.subplots(rows, cols, figsize=(cols*4, rows*3))
axes = axes.flatten()
for i, col in enumerate(categorical_features):
    temp = X.copy()
    temp['target'] = y
    sns.countplot(data=temp, x=col, hue='target', palette='Set1', ax=axes[i])
    axes[i].set_title(f"{col} by target")
    axes[i].legend(loc='upper right', fontsize='small')
for i in range(len(categorical_features), len(axes)):
    fig.delaxes(axes[i])
plt.tight_layout()
plt.show()


# Binary features distribution

In [ ]:
bin_feats = len(binary_features)
cols = 4
rows = (bin_feats + cols - 1) // cols

fig, axes = plt.subplots(rows, cols, figsize=(cols*4, rows*3))
axes = axes.flatten()
for i, col in enumerate(binary_features):
    temp = X.copy()
    temp['target'] = y
    sns.countplot(data=temp, x=col, hue='target', palette='Set1', ax=axes[i])
    axes[i].set_title(f"{col} by target")
    axes[i].legend(loc='upper right', fontsize='small')
for i in range(len(binary_features), len(axes)):
    fig.delaxes(axes[i])
plt.tight_layout()
plt.show()

In [ ]:
# NA check
print(X.isnull().sum())

# feature statistics
display(X.describe())
print(y.value_counts(normalize=True))

In [ ]:
corr = X[numerical_features].corr()
plt.figure(figsize=(8,5))
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title("Numerical Feature Correlation")
plt.show()


# Model

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


In [ ]:
X_train = pd.get_dummies(X_train, columns=categorical_features, dummy_na=True)
X_val = pd.get_dummies(X_val, columns=categorical_features, dummy_na=True)
X_train, X_val = X_train.align(X_val, join='left', axis=1, fill_value=0)

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_val)
y_prob = lr.predict_proba(X_val)[:, 1] 

In [ ]:
plt.hist(y_prob[y_val==0], bins=20, alpha=0.6, label='Benign')
plt.hist(y_prob[y_val==1], bins=20, alpha=0.6, label='Malicious')
plt.xlabel('Predicted Probability')
plt.ylabel('Count')
plt.title('Predicted Probabilities by Class')
plt.legend()
plt.show()



In [ ]:
f1 = f1_score(y_val, y_pred, average='binary')
print("F1-score:", f1)

cm = confusion_matrix(y_val, y_pred)
TN, FP, FN, TP = cm.ravel()

FPR = FP / (FP + TN) if (FP + TN) > 0 else 0
print("False Positive Rate (FPR):", FPR)

In [ ]:
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l2', 'l1'],
    'solver': ['liblinear', 'saga']
}

In [ ]:
lr_grid = LogisticRegression(class_weight='balanced', max_iter=5000, random_state=42)
grid_search = GridSearchCV(
    lr_grid, param_grid, cv=5, scoring='f1', n_jobs=-1
)
grid_search.fit(X_train, y_train)

In [ ]:
print("Best Params:", grid_search.best_params_)
best_lr = grid_search.best_estimator_
y_pred = best_lr.predict(X_val)
y_prob = best_lr.predict_proba(X_val)[:, 1]
print(classification_report(y_val, y_pred))
print('Confusion Matrix:\n', confusion_matrix(y_val, y_pred))
print('ROC-AUC:', roc_auc_score(y_val, y_prob))
print('F1-score:', f1_score(y_val, y_pred))

In [ ]:
cm = confusion_matrix(y_val, y_pred)
TN, FP, FN, TP = cm.ravel()
fpr = FP / (FP + TN) if (FP + TN) > 0 else 0
print('False Positive Rate (FPR):', fpr)

In [ ]:
thresholds = np.arange(0.01, 1, 0.01)
f1s, fprs = [], []

for t in thresholds:
    y_pred_thr = (y_prob >= t).astype(int)
    f1s.append(f1_score(y_val, y_pred_thr))
    cm = confusion_matrix(y_val, y_pred_thr)
    tn, fp, fn, tp = cm.ravel()
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    fprs.append(fpr)

plt.figure(figsize=(8,5))
plt.plot(thresholds, f1s, label='F1-score')
plt.plot(thresholds, fprs, label='FPR')
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.legend()
plt.title('F1-score & FPR vs. Threshold')
plt.show()

In [ ]:
fprs_arr = np.array(fprs)
f1s_arr = np.array(f1s)
thresholds_arr = np.array(thresholds)

idx = np.argmin(np.abs(fprs_arr - 0.05))

print(f"Threshold at closest FPR=0.05: {thresholds_arr[idx]:.2f}")
print(f"Actual FPR: {fprs_arr[idx]:.4f}")
print(f"F1-score: {f1s_arr[idx]:.4f}")

In [ ]:
fprs_arr = np.array(fprs)
f1s_arr = np.array(f1s)
thresholds_arr = np.array(thresholds)

idx = np.argmin(np.abs(f1s_arr - 0.8))

print(f"Threshold at closest FPR=0.05: {thresholds_arr[idx]:.2f}")
print(f"Actual FPR: {fprs_arr[idx]:.4f}")
print(f"F1-score: {f1s_arr[idx]:.4f}")

# SHAP

In [ ]:
X_val = X_val.dropna(axis=1, how='all')
X_train = X_train.dropna(axis=1, how='all')
X_val = X_val.astype(np.float64)
X_train = X_train.astype(np.float64)
X_val.columns = X_val.columns.map(str)
X_train.columns = X_train.columns.map(str)

In [ ]:
import shap

explainer = shap.Explainer(best_lr, X_train)
shap_values = explainer(X_val)

shap.summary_plot(shap_values, X_val, plot_type='bar')

shap.summary_plot(shap_values, X_val)

In [ ]:
coefs = pd.Series(best_lr.coef_.ravel(), index=X_train.columns)
print(coefs.sort_values(ascending=False))


In [ ]:
def summarize_features(df):
    summary = []
    for col in df.columns:
        unique = df[col].nunique(dropna=False)
        non_null = df[col].notnull().sum()
        dtype = df[col].dtype
        if unique == 2:
            ftype = "binary"
        elif unique < 20:
            ftype = "categorical"
        elif np.issubdtype(dtype, np.number):
            ftype = "numerical"
        else:
            ftype = "text/object"
        summary.append({
            "col": col,
            "dtype": dtype,
            "unique": unique,
            "non_null": non_null,
            "auto_type": ftype,
        })
    return pd.DataFrame(summary).sort_values(by='auto_type')

summary = summarize_features(df_pro_large)
display(summary)